In [1]:
import sys
print(sys.version)

3.5.2 (v3.5.2:4def2a2901a5, Jun 26 2016, 10:47:25) 
[GCC 4.2.1 (Apple Inc. build 5666) (dot 3)]


In [2]:
# посмотрим информацию об операционной системе, на которой мы работаем.
import platform
platform.uname()

uname_result(system='Darwin', node='MacAir.local', release='15.5.0', version='Darwin Kernel Version 15.5.0: Tue Apr 19 18:36:36 PDT 2016; root:xnu-3248.50.21~8/RELEASE_X86_64', machine='x86_64', processor='i386')

## Работа с файловой системой

In [75]:
f = open("/tmp/example.txt", "w")
f.write("Технопарк\n")
f.close()

In [76]:
f = open("/tmp/example.txt", "r")
data = f.read()
f.close()
print(data)

Технопарк



In [77]:
# используя context-manager
with open("/tmp/example.txt", "a") as f:
    f.write("МГТУ\n")

In [78]:
!cat /tmp/example.txt

Технопарк
МГТУ


In [79]:
with open("/tmp/example.txt", "r") as f:
    print(f.readlines())

['Технопарк\n', 'МГТУ\n']


In [82]:
# читаем файл по строке, не загружая его полность в память
with open("/tmp/example.txt") as f:
    for line in f:
        print(repr(line))

'Технопарк\n'
'МГТУ\n'


### Директории

In [64]:
import os

os.mkdir("/tmp/park-python")

In [66]:
try:
    os.rmdir("/tmp/park-python")
except IOError as err:
    print(err)

[Errno 2] No such file or directory: '/tmp/park-python'


### stdin, stdout, stderr

In [1]:
import sys
print(sys.stdin)
print(sys.stdout)
print(sys.stderr)

<_io.TextIOWrapper name='<stdin>' mode='r' encoding='UTF-8'>


In [2]:
print(sys.stdin.fileno())

0


In [3]:
print(sys.stdout.fileno())

UnsupportedOperation: IOStream has no fileno.

Так как дескрипторы stdout и stderr переопределены в Jupyter notebook. Давайте посмотрим куда они ведут:

In [4]:
sys.stdout.write("where am I")

where am I

А ведут они как раз в этот ноутбук:)

## Отладка

In [5]:
def find_sum(a, b):
    import pdb
    pdb.set_trace()    
    return a + b

find_sum(1, 4)

> <ipython-input-5-22c1ca74b9d8>(4)find_sum()
-> return a + b
(Pdb) ll
  1  	def find_sum(a, b):
  2  	    import pdb
  3  	    pdb.set_trace()
  4  ->	    return a + b
(Pdb) locals()
{'a': 1, 'b': 4, 'pdb': <module 'pdb' from '/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/pdb.py'>}
(Pdb) c


5

Гораздо удобнее использовать `ipdb`, иногда достаточно просто ф-ии `print()`

## Тестирование

https://pymbook.readthedocs.io/en/latest/testing.html


# Многопоточность

Что такое процесс?

UNIX является многозадачной операционной системой. Это означает, что одновременно может быть запущена более чем одна программа. Каждая программа, работающая в некоторый момент времени, называется процессом.

http://www.kharchuk.ru/%D0%A1%D1%82%D0%B0%D1%82%D1%8C%D0%B8/15-unix-foundations/80-unix-processes

In [60]:
STEPS = 10000000

# Простая программа, складывающая числа.
def worker(steps):
    count = 0
    for i in range(steps):
        count += 1
    return count

print(worker(STEPS))
    
%timeit -n1 worker(STEPS)

print("Напомните преподавателю показать top")

10000000
1 loop, best of 3: 1.37 s per loop
Напомните преподавателю показать top


![title](img/g3.png)

Что такое поток?

Многопоточность является естественным продолжением многозадачности. Каждый из процессов может выполнятся в несколько потоков. Программа выше исполнялась в одном процессе в главном потоке.

http://www.cs.miami.edu/home/visser/Courses/CSC322-09S/Content/UNIXProgramming/UNIXThreads.shtml

Логичный шаг предположить, что 2 потока выполнят программу выше быстрее. Проверим?

In [63]:
import threading
import queue

result_queue = queue.Queue()

STEPS = 10000000
NUM_THREADS = 2

def worker(steps):
    count = 0
    for i in range(steps):
        count += 1
    result_queue.put(count)


def get_count_threaded():    
    count = 0
    threads = []

    for i in range(NUM_THREADS):
        t = threading.Thread(target=worker, args=(STEPS//NUM_THREADS,))
        threads.append(t)
        t.start()

    for i in range(NUM_THREADS):
        count += result_queue.get()

    return count

print(get_count_threaded())
%timeit -n1 get_count_threaded()

10000000
1 loop, best of 3: 1.43 s per loop


![title](img/g4.png)

GIL

https://jeffknupp.com/blog/2012/03/31/pythons-hardest-problem/

Ок. Неужели выхода нет? Есть - multiprocessing

## Мультипроцессинг

In [65]:
import multiprocessing

NUM_PROCESSES = 2
STEPS = 10000000

result_queue = multiprocessing.Queue()

def worker(steps):
    count = 0
    for i in range(steps):
        count += 1

    result_queue.put(count)


def get_count_in_processes():    
    count = 0
    processes = []
    for i in range(NUM_PROCESSES):
        p = multiprocessing.Process(target=worker, args=(STEPS//NUM_PROCESSES,))
        processes.append(t)
        p.start()

    for i in range(NUM_THREADS):
        count += result_queue.get()

    return count

print(get_count_in_processes())
%timeit -n1 get_count_in_processes()

10000000
1 loop, best of 3: 736 ms per loop


**Зачем тогда нужны потоки?**

Все потому что не все задачи CPU-bound. Есть IO-bound задачи, которые прекрасно параллелятся на несколько CPU. Кто приведет пример?

**TODO: примеры**

Чтобы не быть голословными, поднимем HTTP-сервер на порту 8000. По адресу http://localhost:8000 будет отдаваться небольшой кусочек текста. Наша задача - скачивать контент по этому адресу.

In [86]:
import requests

STEPS = 100

def download():
    requests.get("http://127.0.0.1:8000").text

# Простая программа, загружающая контент URL-странички. Типичная IO-bound задача.
def worker(steps):
    for i in range(steps):
        download()

%timeit -n1 worker(STEPS)

1 loop, best of 3: 11.1 s per loop


![title](img/g1.png)

In [93]:
import threading

NUM_THREADS = 64

def worker(steps):
    count = 0
    for i in range(steps):
        download()

def run_worker_threaded():    
    threads = []

    for i in range(NUM_THREADS):
        t = threading.Thread(target=worker, args=(STEPS//NUM_THREADS,))
        threads.append(t)
        t.start()

    for t in threads:
        t.join()

%timeit -n1 run_worker_threaded()

1 loop, best of 3: 452 ms per loop


![title](img/g2.png)

Ради интереса попробуем мультипроцессинг:

In [90]:
import multiprocessing

NUM_PROCESSES = 16

def worker(steps):
    count = 0
    for i in range(steps):
        download()

def run_worker_in_processes():    
    processes = []

    for i in range(NUM_PROCESSES):
        p = multiprocessing.Process(target=worker, args=(STEPS//NUM_PROCESSES,))
        processes.append(p)
        p.start()

    for p in processes:
        p.join()

%timeit -n1 run_worker_in_processes()

1 loop, best of 3: 842 ms per loop


Как мы видим - треды позволили получить лучший результат (Macbook Air 2011 - 64 треда).

Но можем ли мы сделать еще лучше?

* потреблять меньше памяти
* быстрее
* не задумываться о синхронизации потоков

## Асинхронность

In [94]:
import aiohttp
import asyncio

STEPS = 100

async def download():
    async with aiohttp.get("http://127.0.0.1:8000", loop=loop) as response:
        return await response.text()

async def worker(steps, loop):
    await asyncio.gather(*[download() for x in range(steps)])

loop = asyncio.get_event_loop()
%timeit -n1 loop.run_until_complete(worker(STEPS, loop))

1 loop, best of 3: 232 ms per loop


![title](img/g5.png)